## **🚗 Praktikum 1**

Bagging dengan RandomForest untuk mengklasifikasikan jenis tumor

**Import Library**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

kode diatas mengimport library yang akan digunakan untuk praktikum.
1. **`import numpy as np`** digunakan untuk komputasi numerik, contoh array

2. **`import pandas as pd`** digunakan untuk manipulasi data. Ia menyediakan struktur data seperti DataFrame yang sangat berguna untuk mengelola dataset dalam format tabular.

3. **`from sklearn.tree import DecisionTreeClassifier`** untuk klasifikasi dengan memecah dataset berdasarkan fitur yang memberikan informasi paling signifikan.

4. **`from sklearn.ensemble import RandomForestClassifier`** merupakan model ensemble learning yang terdiri dari beberapa decision trees. Hasil akhirnya didasarkan pada voting mayoritas dari hasil prediksi setiap tree. Random forest sering lebih akurat dan tidak mudah overfitting dibandingkan decision tree tunggal.

5. **`from sklearn.model_selection import train_test_split`** untuk membagi dataset menjadi dua bagian: data pelatihan (training data) dan data pengujian (testing data).

6. **`from sklearn.metrics import accuracy_score, classification_report`**:
    - `accuracy_score` mengukur akurasi model
    - `classification_report`menghasilkan laporan metrik klasifikasi yang detail, seperti precision, recall, F1-score, dan support untuk setiap kelas.


**Persiapan Data**

In [ ]:
# Load data
df = pd.read_csv('data/wbc.csv')

df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


Membaca dan menampilkan 5 baris teratas dari data wbc

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


menghitung jumlah baris yang berisikan null pada setiap kolom

In [ ]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
X = df.iloc[:,3:-1]
y = df['diagnosis']
y = y.map({'M':1, 'B':0}) # Encode label

# Cek jumlah fitur dan instance
X.shape

(569, 29)

Kode di atas melakukan seleksi fitur dari `df` dan melakukan encoding pada kolom `diagnosis`.

1. Seleksi Fitur
   ```python
   X = df.iloc[:,3:-1]
   ```
   mengambil subset dari kolom pada DataFrame `df`, mulai dari kolom ke-4 (indeks 3, karena indeks dimulai dari 0) sampai kolom terakhir, kecuali kolom paling akhir (kolom `-1`).

2. Menentukan Label Target
   ```python
   y = df['diagnosis']
   ```
   mengambil kolom `diagnosis` dari DataFrame `df` yang akan digunakan sebagai label target untuk model.

3. Encoding Label
   ```python
   y = y.map({'M': 1, 'B': 0})
   ```
   mengubah nilai dalam kolom `diagnosis` dari label kategorikal menjadi numerik, di mana 'M' (malignant) diberi nilai 1 dan 'B' (benign) diberi nilai 0. Ini penting karena banyak algoritma machine learning hanya bisa bekerja dengan data numerik.

4. Cek Jumlah Fitur dan Instance
   ```python
   X.shape
   ```
   mengecek bentuk dari data fitur `X`, yaitu berapa banyak baris (jumlah instance) dan kolom (jumlah fitur) dalam subset yang telah diambil.

**Split data training dan testing**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

KOde di atas membagi data menjadi 2 bagian yaitu test dan traning dengan perbandingan 0.8 : 0.2

**Traning Decision Tree**

In [ ]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.93
Test set accuracy: 0.9298245614035088


**Decision Tree** bekerja dengan membagi data secara berurutan berdasarkan kriteria tertentu (misalnya Gini impurity).

`dt = DecisionTreeClassifier()` untuk membuat model decision tree dengan parameter default.

`dt.fit(X_train, y_train)` untuk melatih model decision tree menggunakan set pelatihan `X_train` dan label `y_train`.

`y_pred_dt = dt.predict(X_test)` untuk memprediksi label dari set pengujian `X_test`.

`accuracy_score` membandingkan prediksi `y_pred_dt` dengan nilai sebenarnya `y_test` untuk menghitung akurasi model dan`{:.2f}.format(acc_dt)` akan memformat hasil akurasi dalam 2 digit desimal.

**Training RandomForest**

In [ ]:
# Pada kasus kali ini kita akan menggunakan estimator pada RandomForest
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

rf = RandomForestClassifier(n_estimators=10, random_state=1)

# Sesuaikan dt ke set training
rf.fit(X_train, y_train)

# Memprediksi label set test
y_pred_rf = rf.predict(X_test)

#  menghitung set accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")

Test set accuracy: 0.96
Test set accuracy: 0.956140350877193


**Random Forest** mengkombinasikan beberapa decision tree untuk memberikan hasil yang lebih stabil dan akurat, karena memanfaatkan ensembel beberapa model

`rf = RandomForestClassifier(n_estimators=10, random_state=1)` u membuat model random forest dengan 10 decision trees (`n_estimators=10`). Setiap decision tree akan dilatih pada subset data yang berbeda. `random_state=1` digunakan untuk memastikan hasil yang konsisten ketika model dilatih ulang. Secara default, kriteria yang digunakan juga adalah **Gini impurity**.

`rf.fit(X_train, y_train)` melatih model random forest menggunakan set pelatihan `X_train` dan label `y_train`.

`y_pred_rf = rf.predict(X_test)` menggunakan model random forest yang sudah dilatih untuk memprediksi label dari set pengujian `X_test`.

`accuracy_score` membandingkan prediksi `y_pred_rf` dengan nilai sebenarnya `y_test` untuk menghitung akurasi model random forest.


## **🚕 Praktikum 2**

Boosting dengan AdaBoost untuk mengklasifikasikan jenis bunga Iris

**Import Library**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import AdaBoostClassifier # import AdaBoost
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder # Kebutuhan encoding label

Menimport library yang akan digunakan pada praktikum 2, termasuk DT dan Adaboost

**Persiapan Data**

In [ ]:
# Load data
df = pd.read_csv('data/iris.csv')

df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


Membaca dan menampilkan 5 baris teratas dari data iris

In [ ]:
# Cek kolom null
df.isnull().sum()

,0
Id,0
SepalLengthCm,0
SepalWidthCm,0
PetalLengthCm,0
PetalWidthCm,0
Species,0


Menghitung baris yang null pada setiap kolom

In [ ]:
# Seleksi fitur
X = df.iloc[:,2:-1]
y = df['Species']

# encode label
ec = LabelEncoder()
y = ec.fit_transform(y)

# Cek jumlah fitur dan instance
print(X.shape)

# Cek label
print(y)

(150, 3)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


Menyeleksi fitur dimana pada X mengambil subset dari kolom ke 3 hingga kolom terakhir (PetalWidthCm), kecuali kolom paling akhir serta y mengambil kolom species yang berada pada ujung data. Kemudian diencoder untuk label y dan menampilkan hasilya

**Split data training dan testing**

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

Membagi data menjadi training dan testing dengan perbandingan tes size hanya 20% dari keseluruhan data

**Training Decision Tree**

In [ ]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier()

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy
acc_dt = accuracy_score(y_test, y_pred_dt)
print("Test set accuracy: {:.2f}".format(acc_dt))
print(f"Test set accuracy: {acc_dt}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


Decision Tree Classifier menggunakan single tree untuk melakukan prediksi.

`dt = DecisionTreeClassifier()` membuat model Decision Tree menggunakan parameter default, di mana kriteria untuk memisahkan node adalah Gini impurity.

`dt.fit(X_train, y_train)` melatih model decision tree menggunakan data pelatihan `X_train` dan label `y_train`.

`y_pred_dt = dt.predict(X_test)` digunakan untuk memprediksi label pada data pengujian `X_test`.

`acc_dt = accuracy_score(y_test, y_pred_dt)` untuk menghitung akurasi model dengan membandingkan prediksi `y_pred_dt` dengan label sebenarnya `y_test`.

**Training AdaBoost**

In [ ]:
# Pada kasus kali ini kita akan menggunakan estimator pada AdaBoost
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

ada = AdaBoostClassifier(n_estimators=2)

# Sesuaikan dt ke set training
ada.fit(X_train, y_train)

# Memprediksi label set test
y_pred_ada = ada.predict(X_test)

#  menghitung set accuracy
acc_ada = accuracy_score(y_test, y_pred_ada)
print("Test set accuracy: {:.2f}".format(acc_ada))
print(f"Test set accuracy: {acc_ada}")

Test set accuracy: 0.97
Test set accuracy: 0.9666666666666667


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


**AdaBoost Classifier** adalah ensemble model yang meningkatkan akurasi dengan menggabungkan beberapa weak learners. Model ini biasanya lebih kuat dalam menangani overfitting dibandingkan dengan model decision tree tunggal.

`ada = AdaBoostClassifier(n_estimators=2)` membuat model AdaBoost, yang merupakan ensemble method (metode penggabungan beberapa model) dengan parameter `n_estimators=2`, artinya akan menggunakan 2 model weak learners (model yang performanya di bawah standar, biasanya Decision Tree berukuran kecil) secara berurutan untuk meningkatkan performa.

`ada.fit(X_train, y_train)` melatih model AdaBoost pada data pelatihan `X_train` dan label `y_train`. Model ini menggunakan Decision Trees sebagai weak learners secara default.

`y_pred_ada = ada.predict(X_test)` menggunakan model AdaBoost yang sudah dilatih untuk memprediksi label pada set pengujian `X_test`.

`acc_ada = accuracy_score(y_test, y_pred_ada)` digunakan untuk menghitung akurasi model AdaBoost pada data pengujian.

## **🚙 Praktikum 3**

Stacking

Lengkapi bagian berikut dengan data sesuai tugas, dan tentukan perbedaan nilai akurasi antara Random Forest, Adaboost, dan Stacking

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd

In [ ]:
# Load data
df = pd.read_csv('data/diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
# Cek kolom null
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [ ]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
X = df.iloc[:,:-1]
y = df['Outcome']

# Cek jumlah fitur dan instance
X.shape

(768, 8)

In [ ]:
# Assuming 'X' and 'y' are defined, let's split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [ ]:
# Model Random Forest
rf_model = RandomForestClassifier(n_estimators=50, random_state=42)
rf_model.fit(X_train, y_train)
rf_accuracy = rf_model.score(X_test, y_test)

# Model AdaBoost
ada_model = AdaBoostClassifier(n_estimators=50, random_state=42)
ada_model.fit(X_train, y_train)
ada_accuracy = ada_model.score(X_test, y_test)

# Model Stacking
layer_one_estimators = [
    ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('knn_1', KNeighborsClassifier(n_neighbors=5))
]
layer_two_estimators = [
    ('dt_2', DecisionTreeClassifier()),
    ('rf_2', RandomForestClassifier(n_estimators=50, random_state=42)),
]
layer_two = StackingClassifier(estimators=layer_two_estimators, final_estimator=LogisticRegression())

stacking_model = StackingClassifier(estimators=layer_one_estimators, final_estimator=layer_two)
stacking_model.fit(X_train, y_train)
stacking_accuracy = stacking_model.score(X_test, y_test)

# Menampilkan akurasi
print(f'Akurasi Random Forest: {rf_accuracy:.4f}')
print(f'Akurasi Adaboost: {ada_accuracy:.4f}')
print(f'Akurasi Stacking: {stacking_accuracy:.4f}')

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Akurasi Random Forest: 0.7448
Akurasi Adaboost: 0.7344
Akurasi Stacking: 0.7240


## **🚓 Praktikum 4**

Stacking dengan Voting

**Import Library**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB # import Naive Bayes model Gaussian (asumsi data terdistribusi normal)
from sklearn.svm import SVC # import SVM classifier
from sklearn.ensemble import VotingClassifier # import model Voting
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

**Persiapan Data**

In [ ]:
# Load Data

dbt = pd.read_csv('data/diabetes.csv')

dbt.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Membaca data diabetes dan menyimpannya dalam bentuk dataframe lalu meampilkan 5 baris teratas

In [ ]:
# Cek nama kolom
dbt.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

Pengecekan nama-nama kolom yang ada pada data diabetes

In [ ]:
# Cek kolom null
dbt.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


Mengecek data yang nilainya null pada setiap kolom di DataFrame dan menghitung total jumlah baris yang memiliki nilai null

In [ ]:
# Pada kasus ini, agak tidak masuk akal jika beberapa parameter bernilai 0
# sebagai contoh adalah nilai 'Glucose', 'BloodPlessure' ataupun 'Insulin'.
# Sekecil apapun nilainya, setiap manusia yang hidup pasti miliki nilai-nilai tersebut

# Kita akan manipulasi nilai yang 0 dengan melakukan 'imputasi' atau mengganti nilainya dengan nilai sintetis
# Pada kasus ini, kita akan menggunakan nilai mean

# Cek kolom neng nilai 0
feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
for column in feature_columns:
    print("============================================")
    print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")

Pregnancies ==> Missing zeros : 111
Glucose ==> Missing zeros : 5
BloodPressure ==> Missing zeros : 35
SkinThickness ==> Missing zeros : 227
Insulin ==> Missing zeros : 374
BMI ==> Missing zeros : 11
DiabetesPedigreeFunction ==> Missing zeros : 0
Age ==> Missing zeros : 0


`feature_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']` untuk menentukan kolom yang akan dijadikan fitur. Lalu memeriksa daftar kolom yang nilainya nol. Kolom-kolom ini berhubungan dengan kesehatan, dan memiliki nilai nol di antaranya bisa jadi tidak realistis atau menunjukkan data yang hilang.

```python
     for column in feature_columns:
       print("============================================")
       print(f"{column} ==> Missing zeros : {len(dbt.loc[dbt[column] == 0])}")
```

Iterasi ini akan memeriksa setiap kolom dalam `feature_columns`.

`dbt.loc[dbt[column] == 0]`mengambil semua baris dari DataFrame `dbt` di mana nilai pada `column` sama dengan 0.

`len(...)` menghitung jumlah baris yang memiliki nilai 0 untuk kolom tersebut.

In [ ]:
# Impute nilai 0 dengan mean
from sklearn.impute import SimpleImputer

fill_values = SimpleImputer(missing_values=0, strategy="mean", copy=False)

dbt[feature_columns] = fill_values.fit_transform(dbt[feature_columns])

Kode di atas digunakan untuk memanipulasi data, imputasi atau mengganti nilai nol dengan nilai yang lebih realistis, dalam hal ini menggunakan nilai rata-rata (mean) dari kolom tersebut.

- `replace(0, np.nan)` mengganti nilai 0 dengan NaN untuk menghitung mean tanpa memperhitungkan nilai 0

- `.mean()` menghitung nilai rata-rata dari kolom.

- `replace(0, mean_value, inplace=True)` mengganti semua nilai 0 di kolom tersebut dengan nilai rata-rata yang telah dihitung, dengan `inplace=True` untuk memperbarui DataFrame `dbt` secara langsung.


In [ ]:
X = dbt[feature_columns]
y = dbt.Outcome

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Menentukan X (independent) dan y (dependent) dimana X diambil dari feature_columns yang telah dilakukan sebelumnya dan fitur y (label target) merupakan kolom outcomes. Setelah itu data dibagi menjadi train dan test

**Training dengan GaussianNB**

**Standarisasi Fitur**

In [ ]:
# Karena asumsi Gaussian NB adalah data terdistribusi secara normal,
# maka kita perlu melakukan standarisasi

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

Standarisasi pada fitur-fitur yang ada dalam dataset penting dilakukan karena GNB mengasumsikan bahwa data terdistribusi normal.

```python
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)
```

- `StandardScaler()` digunakan untuk menstandarkan fitur dengan cara mengubahnya menjadi distribusi normal (mean = 0 dan variance = 1).
- `fit_transform(X_train)`, `fit_transform` akan menghitung rata-rata dan deviasi standar dari fitur, kemudian melakukan transformasi pada data pelatihan.
- `transform(X_test)`, `transform` hanya akan menerapkan transformasi yang sama (rata-rata dan deviasi standar) yang diperoleh dari `X_train`, tanpa menghitung ulang.


**Training dan Evaluasi**

In [ ]:
# Buat obyek GaussianNB
gnb_std = GaussianNB()

# Fit dengan data yang telah di standarisasi
gnb_std.fit(X_train_std, y_train)

# Prediksi dengan data test
y_pred_gnb = gnb_std.predict(X_test_std)

# Evaluasi akurasi testing data
acc_gnb = accuracy_score(y_test, y_pred_gnb)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_gnb))
print(f"Test set accuracy: {acc_gnb}")

Test set accuracy: 0.74
Test set accuracy: 0.7359307359307359


`gnb_std = GaussianNB()` membuat objek model Gaussian Naive Bayes. Model ini cocok untuk data yang berdistribusi normal.

`gnb_std.fit(X_train_std, y_train)` melatih model GNB menggunakan data pelatihan yang telah distandarisasi.

`y_pred_gnb = gnb_std.predict(X_test_std)` menggunakan model yang telah dilatih untuk memprediksi label pada data pengujian yang juga telah distandarisasi.

`acc_gnb = accuracy_score(y_test, y_pred_gnb)` menghitung akurasi model dengan membandingkan label prediksi `y_pred_gnb` dengan label sebenarnya `y_test`.

**Training dengan SVM Linier**

In [ ]:
# Model SVM linier tanpa tunnning hyperparameter
svm_lin = SVC(kernel='linear')

# Fit ke model
svm_lin.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_lin = svm_lin.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_lin = accuracy_score(y_test, y_pred_svm_lin)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_lin))
print(f"Test set accuracy: {acc_svm_lin}")

Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


- `SVC(kernel='linear')` membuat model Support Vector Classifier dengan kernel linear. Kernel linear cocok untuk data yang dapat dipisahkan secara linier.

- `.fit(X_train_std, y_train)` melatih model dengan data pelatihan yang telah distandarisasi.

- `.predict(X_test_std)` menghasilkan prediksi untuk data pengujian yang telah distandarisasi.

- `accuracy_score(y_test, y_pred_svm_lin)` menghitung akurasi dari prediksi model SVM linear terhadap data pengujian.



**Training dengan SVM RBF**

In [ ]:
# Model SVM RBF tanpa tunnning hyperparameter
svm_rbf = SVC(kernel='rbf')

# Fit ke model
svm_rbf.fit(X_train_std, y_train)

# Prediksi
y_pred_svm_rbf = svm_rbf.predict(X_test_std)

# Evaluasi akurasi testing data
acc_svm_rbf = accuracy_score(y_test, y_pred_svm_rbf)

# Print hasil evaluasi
print("Test set accuracy: {:.2f}".format(acc_svm_rbf))
print(f"Test set accuracy: {acc_svm_rbf}")

Test set accuracy: 0.72
Test set accuracy: 0.7229437229437229


SVC(kernel='rbf')nModel SVM dengan kernel Radial Basis Function (RBF). Kernel RBF dapat menangani data yang tidak dapat dipisahkan secara linier.

- Proses pelatihan, prediksi, dan evaluasi akurasi dilakukan sama seperti pada model SVM linear.

**Training dengan Voting**

In [ ]:
# Definisikan algoritma yang akan digunakan untuk voting

clf1 = GaussianNB()
clf2 = SVC(kernel='linear')
clf3 = SVC(kernel='rbf', probability=True)

# model hard voting
voting = VotingClassifier(estimators=[('GaussianNB', clf1), ('SVM-LIN', clf2), ('SVM-RBF', clf3)], voting='hard')

# Fit model
voting.fit(X_train_std, y_train)

# Prediksi
y_pred_vt1 = voting.predict(X_test_std)

# Evaluasi akurasi testing data
acc_vt1 = accuracy_score(y_test, y_pred_vt1)

# Print hasil evaluasi
print('Voting Hard')
print("Test set accuracy: {:.2f}".format(acc_vt1))
print(f"Test set accuracy: {acc_vt1}")

Voting Hard
Test set accuracy: 0.74
Test set accuracy: 0.7402597402597403


- **Definisikan Estimator**:
  - `clf1`, `clf2`, dan `clf3` kode yang menciptakan tiga model: Gaussian Naive Bayes, SVM linear, dan SVM RBF.
  - **`probability=True`** pada SVM RBF mengaktifkan estimasi probabilitas, yang mungkin diperlukan jika menggunakan voting soft (tidak berlaku untuk hard voting).
  
- **VotingClassifier** menggabungkan beberapa model untuk melakukan voting.
  - `voting='hard'` Menggunakan voting keras, di mana prediksi akhir adalah label kelas yang paling sering muncul di antara semua model.
  
- `voting.fit(X_train_std, y_train)` melatih model voting menggunakan data pelatihan yang telah distandarisasi.

- `voting.predict(X_test_std)` menghasilkan prediksi untuk data pengujian menggunakan model voting.

- `accuracy_score` menghitung akurasi yang kemudian akan dicetak hasil evaluasinya.

## **🤩 Tugas**

**Tugas 1**

Terdapat dataset mushroom. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik

In [36]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  # Import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [37]:
df = pd.read_csv('data/mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [38]:
# Label encoding untuk setiap kolom
for column in df.columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

X = df.iloc[:,1:]
y = df['class']

# Cek hasil encoding
print(df.head())

   class  cap-shape  cap-surface  cap-color  bruises  odor  gill-attachment  \
0      1          5            2          4        1     6                1   
1      0          5            2          9        1     0                1   
2      0          0            2          8        1     3                1   
3      1          5            3          8        1     6                1   
4      0          5            2          3        0     5                1   

   gill-spacing  gill-size  gill-color  ...  stalk-surface-below-ring  \
0             0          1           4  ...                         2   
1             0          0           4  ...                         2   
2             0          0           5  ...                         2   
3             0          1           5  ...                         2   
4             1          0           4  ...                         2   

   stalk-color-above-ring  stalk-color-below-ring  veil-type  veil-color  \
0         

In [39]:
# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shape of training and testing data
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6499, 22), (1625, 22), (6499,), (1625,))

Desicion Tree

In [42]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

In [44]:
# Define the parameter grid to tune the hyperparameters
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid_dt, cv=5)

grid_search.fit(X_train, y_train)
best_dtree_reg = grid_search.best_estimator_
y_pred = best_dtree_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5

accuracy = accuracy_score(y_test, y_pred)
best_params = grid_search.best_params_

print(f"Best parameters: {best_params}")
print(f"Test RMSE: {rmse}")
print(f"Test set accuracy: {accuracy:.2f}")

Best parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_split': 2}
Test RMSE: 0.0
Test set accuracy: 1.00


Random Forest

In [55]:
param_grid = {
    'n_estimators': [50, 100, 150],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [57]:
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

model_grid = grid_search.best_estimator_
model_grid.fit(X_train, y_train)
y_pred_rf = model_grid.predict(X_test)

rf_accuracy = accuracy_score(y_test, y_pred_rf)

In [59]:
print("Best Random Forest Parameters:", grid_search.best_params_)
print("Random Forest Accuracy:", rf_accuracy)
print(classification_report(y_pred_rf, y_test))

Best Random Forest Parameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
Random Forest Accuracy: 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       843
           1       1.00      1.00      1.00       782

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



**Tugas 2**

Terdapat dataset mushroom. Berdasarkan dataset tersebut, bandingkan peforma antara algoritma Decision Tree dan AdaBoost. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder  # Import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('data/mushrooms.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [ ]:
# Label encoding untuk setiap kolom
for column in df.columns:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])

X = df.iloc[:,1:]
y = df['class']

In [ ]:
# Split the data into training and testing sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check shape of training and testing data
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6499, 22), (1625, 22), (6499,), (1625,))

In [ ]:
# Decision Tree Classifier
dt = DecisionTreeClassifier(random_state=42)

# Hyperparameter tuning untuk Decision Tree
param_grid_dt = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None] + list(range(1, 11)),
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

grid_search_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_dt.fit(X_train, y_train)

# Hasil tuning terbaik Decision Tree
best_dt = grid_search_dt.best_estimator_
y_pred_dt = best_dt.predict(X_test)
dt_accuracy = accuracy_score(y_test, y_pred_dt)

print(f"Best Decision Tree Params: {grid_search_dt.best_params_}")
print(f"Decision Tree Accuracy: {dt_accuracy}")

Best Decision Tree Params: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2}
Decision Tree Accuracy: 1.0


In [ ]:
# AdaBoost Classifier
ada = AdaBoostClassifier(estimator=DecisionTreeClassifier(), random_state=42)

# Hyperparameter tuning untuk AdaBoost
param_grid_ada = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.01, 0.1, 1],
    'estimator__max_depth': [1, 2, 3],
    'estimator__min_samples_split': [2, 5, 10]
}

grid_search_ada = GridSearchCV(ada, param_grid_ada, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_ada.fit(X_train, y_train)

# Hasil tuning terbaik AdaBoost
best_ada = grid_search_ada.best_estimator_
y_pred_ada = best_ada.predict(X_test)
ada_accuracy = accuracy_score(y_test, y_pred_ada)

print(f"Best AdaBoost Params: {grid_search_ada.best_params_}")
print(f"AdaBoost Accuracy: {ada_accuracy}")

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Best AdaBoost Params: {'estimator__max_depth': 1, 'estimator__min_samples_split': 2, 'learning_rate': 1, 'n_estimators': 50}
AdaBoost Accuracy: 1.0


In [ ]:
# Perbandingan hasil
print(f"Decision Tree Accuracy: {dt_accuracy}")
print(f"AdaBoost Accuracy: {ada_accuracy}")

Decision Tree Accuracy: 1.0
AdaBoost Accuracy: 1.0


**Tugas 3**

Dengan menggunakan dataset diabetes, buatlah ensemble voting dengan algoritma

1.   Logistic Regression

2.   SVM kernel polynomial

3.   Decission Tree

Anda boleh melakukan eksplorasi dengan melakukan tunning hyperparameter


In [29]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Load data
df = pd.read_csv('data/diabetes.csv')

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
# Cek kolom null
df.isnull().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


In [30]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
X = df.iloc[:,:-1]
y = df['Outcome']

# Cek jumlah fitur dan instance
X.shape

(768, 8)

In [31]:
# Assuming 'X' and 'y' are defined, let's split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [24]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

# Standarisasi pada fitur di X_train dan X_test
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [35]:
# Membangun pipeline untuk menyekal data dan melakukan Logistic Regression
log_reg = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

# Mendefinisikan hyperparameter untuk GridSearch
log_params = {
    'logisticregression__C': [0.01, 0.1, 1, 10, 100],
    'logisticregression__solver': ['liblinear', 'lbfgs']
}

# Melakukan GridSearchCV
log_grid_search = GridSearchCV(log_reg, log_params, cv=5, scoring='accuracy')
log_grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
print("Best Logistic Regression Parameters:", log_grid_search.best_params_)

# Mendapatkan model terbaik
best_model = log_grid_search.best_estimator_

# Melakukan prediksi pada data uji
y_pred_log = best_model.predict(X_test)

# Menghitung akurasi dengan accuracy_score
acc_log = accuracy_score(y_test, y_pred_log)

# Menampilkan hasil akurasi
print(f"Accuracy of Logistic Regression: {acc_log:.4f}")


Best Logistic Regression Parameters: {'logisticregression__C': 0.1, 'logisticregression__solver': 'lbfgs'}
Accuracy of Logistic Regression: 0.7344


In [26]:
from sklearn.metrics import accuracy_score

# Tuning hyperparameter untuk SVM
svm_params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'degree': [2, 3, 4],
    'kernel': ['poly']
}

svm = SVC()
svm_grid_search = GridSearchCV(svm, svm_params, cv=5, scoring='accuracy')
svm_grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
print("Best SVM Parameters:", svm_grid_search.best_params_)

# Mendapatkan model terbaik
best_svm_model = svm_grid_search.best_estimator_

# Melakukan prediksi pada data uji
y_pred_svm = best_svm_model.predict(X_test)

# Menghitung akurasi dengan accuracy_score
acc_svm = accuracy_score(y_test, y_pred_svm)

# Menampilkan hasil akurasi
print(f"Accuracy of SVM: {acc_svm:.4f}")


Best SVM Parameters: {'C': 10, 'degree': 3, 'kernel': 'poly'}
Accuracy of SVM: 0.7448


In [32]:
from sklearn.metrics import accuracy_score

# Tuning hyperparameter untuk Decision Tree
dt_params = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None] + list(range(1, 11)),
    'min_samples_split': [2, 5, 10]
}

dt_classifier = DecisionTreeClassifier()
dt_grid_search = GridSearchCV(dt_classifier, dt_params, cv=5, scoring='accuracy')
dt_grid_search.fit(X_train, y_train)

# Menampilkan parameter terbaik
print("Best Decision Tree Parameters:", dt_grid_search.best_params_)

# Mendapatkan model terbaik
best_dt_model = dt_grid_search.best_estimator_

# Melakukan prediksi pada data uji
y_pred_dt = best_dt_model.predict(X_test)

# Menghitung akurasi dengan accuracy_score
acc_dt = accuracy_score(y_test, y_pred_dt)

# Menampilkan hasil akurasi
print(f"Accuracy of Decision Tree: {acc_dt:.4f}")

Best Decision Tree Parameters: {'criterion': 'gini', 'max_depth': 3, 'min_samples_split': 2}
Accuracy of Decision Tree: 0.7135


In [33]:
# Menggunakan model terbaik dari grid search
best_log_reg = log_grid_search.best_estimator_
best_svm = svm_grid_search.best_estimator_
best_dt = dt_grid_search.best_estimator_

# Membuat ensemble voting classifier
voting_classifier = VotingClassifier(estimators=[
    ('log_reg', best_log_reg),
    ('svm', best_svm),
    ('decision_tree', best_dt)
], voting='hard')

# Melatih model ensemble
voting_classifier.fit(X_train, y_train)

# Evaluasi akurasi
y_pred = voting_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Voting Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

Ensemble Voting Accuracy: 0.7447916666666666
              precision    recall  f1-score   support

           0       0.77      0.87      0.82       125
           1       0.68      0.51      0.58        67

    accuracy                           0.74       192
   macro avg       0.72      0.69      0.70       192
weighted avg       0.74      0.74      0.73       192

